In [4]:
# data preparation
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
data=pd.read_csv("C:\\Users\\jimen\\EMBEDED LEARN\\ipl dataset\\deliveries1.csv")
data1=pd.DataFrame(data)

data1["score"]=data1.groupby(["match_id","inning"])["total_runs"].cumsum()

data1.rename(columns={"total_runs":"run_per_ball"},inplace=True)
data1["crr"]=data1["score"]/((data1["over"]-1)+data1["ball"])
matchdf=pd.read_csv("C:\\Users\\jimen\\EMBEDED LEARN\\ipl dataset\\matches1.csv")
matchdf=matchdf.drop(["city","player_of_match"],axis=1)
matchdf["winner"]=matchdf["winner"].fillna("no result")
data1=pd.merge(data1,matchdf,on=["match_id"],how="left")
data1.drop(["match_id","run_per_ball"],axis=1,inplace=True)
y=data1.iloc[:,13]
x=data1.iloc[:,:13]
print(data1)





        inning         batting_team                 bowling_team  over  ball  \
0            1  Sunrisers Hyderabad  Royal Challengers Bangalore     1     1   
1            1  Sunrisers Hyderabad  Royal Challengers Bangalore     1     2   
2            1  Sunrisers Hyderabad  Royal Challengers Bangalore     1     3   
3            1  Sunrisers Hyderabad  Royal Challengers Bangalore     1     4   
4            1  Sunrisers Hyderabad  Royal Challengers Bangalore     1     5   
...        ...                  ...                          ...   ...   ...   
179073       2  Chennai Super Kings               Mumbai Indians    20     2   
179074       2  Chennai Super Kings               Mumbai Indians    20     3   
179075       2  Chennai Super Kings               Mumbai Indians    20     4   
179076       2  Chennai Super Kings               Mumbai Indians    20     5   
179077       2  Chennai Super Kings               Mumbai Indians    20     6   

          batsman non_striker      bowl

In [5]:
# model preparation decision tree
dt=DecisionTreeClassifier()
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3)
dt




DecisionTreeClassifier()

In [6]:
#logistic regression
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(solver='liblinear')
lr

LogisticRegression(solver='liblinear')

In [7]:
#random forest
from sklearn.ensemble import RandomForestClassifier
rf_model=RandomForestClassifier(n_estimators=100, max_depth=10)
rf_model


RandomForestClassifier(max_depth=10)

In [8]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators=[
    ('rf_model', rf_model), ('lr', lr),('dt',dt)], voting='hard')
voting_clf


VotingClassifier(estimators=[('rf_model', RandomForestClassifier(max_depth=10)),
                             ('lr', LogisticRegression(solver='liblinear')),
                             ('dt', DecisionTreeClassifier())])

In [9]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
    ('voting_model', voting_clf)
])
pipeline

Pipeline(steps=[('encoder', OneHotEncoder(handle_unknown='ignore')),
                ('voting_model',
                 VotingClassifier(estimators=[('rf_model',
                                               RandomForestClassifier(max_depth=10)),
                                              ('lr',
                                               LogisticRegression(solver='liblinear')),
                                              ('dt',
                                               DecisionTreeClassifier())]))])

In [10]:
#training the model
pipeline.fit(xtrain,ytrain)

Pipeline(steps=[('encoder', OneHotEncoder(handle_unknown='ignore')),
                ('voting_model',
                 VotingClassifier(estimators=[('rf_model',
                                               RandomForestClassifier(max_depth=10)),
                                              ('lr',
                                               LogisticRegression(solver='liblinear')),
                                              ('dt',
                                               DecisionTreeClassifier())]))])

In [11]:
#testing
ypred=pipeline.predict(xtest)


In [12]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(ytest, ypred)
accuracy

0.9502643139006776

In [13]:
#loading into flask
import pickle
with open('model.pkl','wb') as file:
    pickle.dump(pipeline,file)


In [26]:
data1.to_csv("ipl_newdata.csv",index=False)

In [25]:
l=data1["toss_decision"].unique().tolist()
print(l)

['field', 'bat']
